# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import sys
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, make_scorer
import re  



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [24]:
print(f"DATABASE: {'DisasterResponse.db'}")

DATABASE: DisasterResponse.db


In [1]:
def load_data(database_filepath):
    """
    Load data from SQLite database.
    
    Args:
    database_filepath: str. Filepath for the SQLite database.
    
    Returns:
    X: DataFrame. Features dataset.
    Y: DataFrame. Labels dataset.
    category_names: list. List of category names.
    """
    engine = create_engine(f'sqlite:///{database_filepath}')
    df = pd.read_sql_table('MessagesCategories', engine)
    
    # Replace inf values with NaN and drop rows with NaN values
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna()

    X = df['message']
    Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)
    category_names = Y.columns.tolist()
    return X, Y, category_names

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    """
    Function to tokenize text.
    """
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens=[]
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

In [4]:
# Here I would like to test the tokenaiztation 
example_text = "This is an example sentence, testing the tokenization."
tokens = tokenize(example_text)
print(tokens)

['this', 'is', 'an', 'example', 'sentence', ',', 'testing', 'the', 'tokenization', '.']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [2]:
# Define multioutput accuracy function
def multioutput_accuracy_score(y_true, y_pred):
    """
    Compute multioutput accuracy.
    
    Args:
    y_true: array-like of shape (n_samples, n_outputs)
        True values for predictions.
    y_pred: array-like of shape (n_samples, n_outputs)
        Predicted values.
        
    Returns:
    accuracy: float
        Average accuracy across all outputs.
    """
    return np.mean(np.equal(y_true, y_pred))

In [3]:
# Define function to build the model
def build_model():
    """
    Build a machine learning pipeline with GridSearchCV.
    
    Returns:
    model: GridSearchCV. Grid search model object.
    """
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),  
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    parameters = {
        'clf__estimator__n_estimators': [5]
    }

    scorer = make_scorer(multioutput_accuracy_score)
    
    model = GridSearchCV(pipeline, param_grid=parameters, scoring=scorer, verbose=2, n_jobs=-1)

    return model

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),  # To tokenize text using CountVectorizer method
    ('tfidf', TfidfTransformer()),  # Apply transformation
    ('clf', MultiOutputClassifier(RandomForestClassifier()))  # Multi-output classifier using RandomForestClassifier
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def f1_pre_acc_evaluation(y_true, y_pred):
    """
    Function to evaluate F1 score, precision, and accuracy.
    """
    # Calculate F1 score, precision, and accuracy
    f1_score = classification_report(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)
    
    # Return a formatted report
    report = f"F1 Score:\n{f1_score}\nPrecision: {precision}\nAccuracy: {accuracy}"
    
    return report

In [9]:
# Make predictions
y_pred = pipeline.predict(X_test)

# Convert predictions to a DataFrame
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

# Generate report
report = f1_pre_acc_evaluation(y_test, y_pred_df)
print(report)

F1 Score:
             precision    recall  f1-score   support

          0       0.82      0.93      0.87      3959
          1       0.80      0.37      0.51       902
          2       0.00      0.00      0.00        25
          3       0.75      0.52      0.62      2156
          4       0.75      0.07      0.13       431
          5       0.74      0.10      0.17       264
          6       0.90      0.06      0.11       151
          7       0.00      0.00      0.00       106
          8       0.72      0.07      0.13       175
          9       0.00      0.00      0.00         0
         10       0.86      0.24      0.38       344
         11       0.79      0.43      0.56       586
         12       0.84      0.23      0.36       487
         13       0.89      0.10      0.18        79
         14       0.71      0.04      0.07       131
         15       1.00      0.02      0.04        55
         16       0.43      0.02      0.04       160
         17       0.87      0.15   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# Get parameters for the pipeline model
params = pipeline.get_params()

In [11]:
# Print the parameters
print("Parameters for the pipeline model:")
print(params)

Parameters for the pipeline model:
{'memory': None, 'steps': [('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenize at 0x7c83505e0048>, vocabulary=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=

In [13]:
from sklearn.metrics import make_scorer, accuracy_score
# Define the scorer
scorer = make_scorer(accuracy_score)

# Define the reduced parameters for grid search
parameters = {
    'clf__estimator__n_estimators' : [5]
}

# Create GridSearchCV object
cv = GridSearchCV(pipeline, param_grid=parameters, scoring=scorer, verbose=2, n_jobs=-1)

In [14]:
# Fit the model
cv.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] clf__estimator__n_estimators=5 ..................................
[CV] clf__estimator__n_estimators=5 ..................................
[CV] clf__estimator__n_estimators=5 ..................................
[CV] ................... clf__estimator__n_estimators=5, total= 1.5min
[CV] ................... clf__estimator__n_estimators=5, total= 1.5min
[CV] ................... clf__estimator__n_estimators=5, total= 1.6min


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.9min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(accuracy_score), verbose=2)

In [15]:
# Get the best parameters and best score
best_params = cv.best_params_
best_score = cv.best_score_

In [16]:
print("Best Parameters:")
print(best_params)
print("\nBest Score:")
print(best_score)

Best Parameters:
{'clf__estimator__n_estimators': 5}

Best Score:
0.175583517433


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
# Evaluate the model
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    for column in y_test.columns:
        print(column)
        print(classification_report(y_test[column], y_pred[:, y_test.columns.get_loc(column)]))
    return y_pred

def f1_pre_acc_evaluation(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)
    report = {
        'F1 Score': f1,
        'Precision': precision,
        'Accuracy': accuracy
    }
    return report

In [18]:
# Evaluate the model
y_pred = evaluate_model(cv, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.60      0.31      0.41      1247
          1       0.81      0.94      0.87      3959

avg / total       0.76      0.79      0.76      5206

request
             precision    recall  f1-score   support

          0       0.89      0.96      0.93      4304
          1       0.71      0.43      0.54       902

avg / total       0.86      0.87      0.86      5206

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5181
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      5206

aid_related
             precision    recall  f1-score   support

          0       0.74      0.80      0.77      3050
          1       0.69      0.61      0.65      2156

avg / total       0.72      0.72      0.72      5206

medical_help
             precision    recall  f1-score   support

          0       0.93      0.99      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
from sklearn.metrics import classification_report, f1_score, precision_score, accuracy_score
# Calculate and print overall metrics
overall_report = f1_pre_acc_evaluation(y_test, y_pred)
print("Overall Metrics:")
for metric, score in overall_report.items():
    print(f"{metric}: {score:.4f}")

Overall Metrics:
F1 Score: 0.5364
Precision: 0.6679
Accuracy: 0.1806


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Based on shown result above, here is an interpretation of the results:
- F1 Score: A score of (0.5364) suggests that the balance between precision and recall is moderate but could be improved.

- Precision: A precision of (0.6679) means that approximately 67% of the model's positive predictions are correct.

- Accuracy: An accuracy of (0.1806) suggests that the model correctly classifies about 18% of the instances, which is relatively low and indicates room for significant improvement.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [21]:
# Save the best model from GridSearchCV as a pickle file
with open('disaster_response_model.pkl', 'wb') as file:
    pickle.dump(cv.best_estimator_, file)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.